### Convert IAPD (Molspin Files) to Magic Format

####Dan Thallner 2023- This notebook takes IAPD formatted files from UF Molspin and converts them for use in PmagPy

In [4]:
# import required packages
import os
import pandas as pd

In [7]:
# Start new Magic contribution for locality

sites_dict = {'site': [], 'method_codes': [], 'citations': []}
samples_dict = {'site': [], 'sample': [],'method_codes': [], 'citations': [],
                'azimuth': [], 'dip': [], 'bed_dip_direction': [], 'bed_dip': []}
specimens_dict = {'specimen': [], 'sample': [],'method_codes': [],'experiments': [],'volume': [], 'citations': []}
measurement_dict = {'specimen': [],'method_codes': [],'dir_dec': [],'dir_inc': [],'magn_volume': [], 'experiment': [],
                    'dir_csd': [],'treat_temp':[], 'meas_temp': [],'treat_ac_field': [], 'citations': [],
                    'instrument_codes': [],'sequence':[], 'magn_moment': [], 'measurement': []}

### Molspin data files:

In [8]:
# add measurement data for a site 
# run this cell once for each directory specified below. Use one directory per site & experiment
# bedding dirs, volume unit, intensity unit, a95 == csd?, treat AC field units


site_name = 'I1936' # Specify name of site
treatment = 'THD' # Specify if the experiment was thermal demag ('THD') or AF demag ('AFD')
site_dir_path = r'h:\Magic2023\I1936th' # absolute or relative path to directory 

#---------------------------------------------------------------------------------------------

duplicate_site = False
if not site_name in sites_dict['site']: sites_dict['site'].append(site_name)
else: duplicate_site = True
if not duplicate_site: sites_dict['citations'].append('This study')
if treatment == 'THD': 
    if not duplicate_site: 
        sites_dict['method_codes'].append('LP-DIR-T')
    else: 
        for j in range(len(sites_dict['site'])):
            if sites_dict['site'] == site_name:
                if not 'LP-DIR-T' in sites_dict['method_codes'][j]:
                    sites_dict['method_codes'][j] += ':LP-DIR-T'
elif treatment == 'AFD': 
    if not duplicate_site: 
        sites_dict['method_codes'].append('LP-DIR-AF')
    else: 
        for j in range(len(sites_dict['site'])):
            if sites_dict['site'] == site_name:
                if not 'LP-DIR-AF' in sites_dict['method_codes'][j]:
                    sites_dict['method_codes'][j] += ':LP-DIR-AF'
else: print('Currently, only THD (thermal demag - directions) or AFD (AF demag - directions) are availabe options')

if not site_dir_path.endswith('/'): site_dir_path += '/'
for root, dirs, files in os.walk(site_dir_path):
    for file in files:
        with open(root+os.sep+file) as specimen_file:
             measurement_data = specimen_file.readlines()
        for i in range(len(measurement_data)):
            if i == 0:
                samples_dict['site'].append(site_name)
                samples_dict['citations'].append('This study')
                specimens_dict['citations'].append('This study')
                header = measurement_data[i].split()
                name = header[0]
                if treatment == 'THD': 
                    samples_dict['method_codes'].append('LP-DIR-T')
                    specimens_dict['method_codes'].append('LP-DIR-T')
                    specimens_dict['experiments'].append(name+'-'+'LP-DIR-T')
                elif treatment == 'AFD': 
                    samples_dict['method_codes'].append('LP-DIR-AF')
                    specimens_dict['method_codes'].append('LP-DIR-AF')
                    specimens_dict['experiments'].append(name+'-'+'LP-DIR-AF')
                samples_dict['sample'].append(name)
                specimens_dict['sample'].append(name)
                specimens_dict['specimen'].append(name)
                bearing = header[1]
                samples_dict['azimuth'].append(bearing)
                plunge = header[2]
                samples_dict['dip'].append(str(float(plunge)-90))
                strike = header[3]
                samples_dict['bed_dip_direction'].append(str(float(strike)+90))
                dip = header[4]
                samples_dict['bed_dip'].append(dip)
                volume = header[5]
                specimens_dict['volume'].append(str(float(volume)/10**6)) # m^3
            else:
                measurement = measurement_data[i].split()
                measurement_dict['sequence'].append(str(i+1))
                measurement_dict['citations'].append('This study')
                measurement_dict['specimen'].append(name)
                treat = measurement[0]
                if treatment == 'THD':
                    measurement_dict['treat_temp'].append(str(float(treat)+273)) # K
                    measurement_dict['meas_temp'].append(str(273)) # K
                    measurement_dict['treat_ac_field'].append(str(0)) # T
                    measurement_dict['method_codes'].append('LT-T-Z')
                    measurement_dict['experiment'].append(name+'-'+'LP-DIR-T')
                    measurement_dict['measurement'].append(name+'-'+'LP-DIR-T'+'-'+str(i))
                    
                elif treatment == 'AFD':
                    measurement_dict['treat_temp'].append(str(273)) # K
                    measurement_dict['meas_temp'].append(str(273)) # K
                    measurement_dict['treat_ac_field'].append(str(float(treat)/10**3)) # T
                    measurement_dict['method_codes'].append('LT-AF-Z')
                    measurement_dict['experiment'].append(name+'-'+'LP-DIR-AF')
                    measurement_dict['measurement'].append(name+'-'+'LP-DIR-AF'+'-'+str(i))
                
                intensity = measurement[1] # mA/m
                measurement_dict['magn_volume'].append(str(float(intensity)/10**3)) # A/m
                measurement_dict['magn_moment'].append(str((float(intensity)/10**3)*(float(volume)/10**6))) # Am^2
                GDec = (measurement[2]) # not needed for demag GUI
                GInc = (measurement[3]) # not needed for demag GUI
                a95 = measurement[4]
                measurement_dict['dir_csd'].append(a95)
                CDec = measurement[5]
                measurement_dict['dir_dec'].append(CDec)
                CInc = measurement[6]
                measurement_dict['dir_inc'].append(CInc)
                measurement_dict['instrument_codes'].append('Molspin')

print('Specimens for site', site_name, 'added successfully')

Specimens for site I1936 added successfully


### Write MagIC formatted files

Run cell after all experiments of the current location have been added using the cell above 

In [9]:
# save files for demag_gui

with open('./sites.txt', 'w') as sites_file:
    sites_file.write('tab\tsites\n')
sites_df = pd.DataFrame(sites_dict)
sites_df.to_csv('./sites.txt',mode='a',index=False,sep='\t')
    
with open('./samples.txt', 'w') as samples_file:
    samples_file.write('tab\tsamples\n')
samples_df = pd.DataFrame(samples_dict)
samples_df.to_csv('./samples.txt',mode='a',index=False,sep='\t')

with open('./specimens.txt', 'w') as specimens_file:
    specimens_file.write('tab\tspecimens\n')
specimens_df = pd.DataFrame(specimens_dict)
specimens_df.to_csv('./specimens.txt',mode='a',index=False,sep='\t')

with open('./measurements.txt', 'w') as measurements_file:
    measurements_file.write('tab\tmeasurements\n')
measurement_df = pd.DataFrame(measurement_dict)
measurement_df.to_csv('./measurements.txt',mode='a',index=False,sep='\t')

print('Files for demag GUI saved to', os.getcwd())

Files for demag GUI saved to C:\Users\jmeer\PmagPy
